In [ ]:
############# Install the dependencies ##################
# !pip install datasets transformers evaluate sacrebleu

####   if code fails below while creating the training arguments , run the following and restart the kernel   #######
# !pip install --upgrade accelerate


# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

#### Reading the dataset from repository

In [ ]:
# ## Import the dataset on which we want to fine-tune the model
# # I have used ai4bharat/samanantar, don't forget to look for the subsets of the dataset and load it accordingly
# from datasets import load_dataset
# odia=load_dataset("ai4bharat/samanantar","or")

In [ ]:
# print(odia)
# Our dataset has three features , src has english text while tgt has odia tranlations of it

In [ ]:
##Import Tokenizer
from transformers import AutoTokenizer

checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# odia["train"]['src']

In [ ]:
# ## Tokenization function
def preprocess(odia):
    # putting all the English sentences into the imput list with the prefix.
    # and the odia translations into the target into the target list.
    inputs = [text for text in odia["src"]]
    target = [text for text in odia["tgt"]]

    model_inputs = {
        "id": odia["idx"],
        "translation": target,  # assign 'translation' directly to 'target'
        "input_ids": [],
        "attention_mask": [],
        "labels": [],
    }

    try:
        # Tokenize all inputs and targets at once, which is more efficient
        tokenized_samples = tokenizer(inputs, max_length=128, truncation=True)
        labels = tokenizer(target, max_length=128, truncation=True)

        # Assign the tokenized samples directly to 'input_ids', 'attention_mask', and 'labels'
        model_inputs["input_ids"] = tokenized_samples["input_ids"]
        model_inputs["attention_mask"] = tokenized_samples["attention_mask"]
        model_inputs["labels"] = labels["input_ids"]

    except Exception as e:
        print(f"Error occurred: {e}")

    return model_inputs

In [ ]:
# Applying the tokenization funtion on the dataset, with batch processing
tokenized_odia = odia.map(preprocess, batched=True)

In [ ]:
# after tokenization , the dataset
print(tokenized_odia)

In [ ]:
# importing the data collators which creates the batches of inputs to be fed to the seq2seq model.
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model="facebook/nllb-200-distilled-600M"
)

In [ ]:
# Importing sacrebleu to evaluate the blue score of the model.
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
# funtion to calculate and process the blue score
import numpy as np


def postprocess(pred, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
# trying to create a trainer using Pytorch
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
# tuning the hyperparameters of Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="fine_tune_model",
    report_to="wandb",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    eval_accumulation_steps=50,
    gradient_accumulation_steps=4,
    predict_with_generate=True,
    # push_to_hub=True,
)
# passing the arguments to trainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_odia["train"],
    eval_dataset=tokenized_odia["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model("/home/jupyter/notebooks/notebook/fine-tune/")